In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, :, :])
        return out

In [3]:
# Hyperparameters
num_classes = 32
num_epochs = 300
batch_size = 2048
input_size = 1
model_dir = 'model'
window_size = 10
log = 'add_padding_batch_size={}_epoch={}_window_size={}'.format(str(batch_size), str(num_epochs),str(window_size))
num_layers = 2
hidden_size = 64
file_dir = 'data/'
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [5]:
model.load_state_dict(torch.load(model_dir + '/' + log + '.pt'))
model.to(device)
model.eval()

Model(
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=32, bias=True)
)

In [6]:
import random

In [7]:
def generate_seq(start,window_size=10,num_candidates=5,scope=None):
    if isinstance(start,list):
        start = torch.FloatTensor(start).reshape(1,-1)
    bg = start.size(1) 
    if scope==None:
        scope=num_candidates
    for i in range(bg,bg+window_size):
#         start = torch.FloatTensor(start)
        seq = start.clone().detach().view(-1, i, input_size).to(device)
        output = model(seq).cpu()[:,-1,:]
        output = output.reshape(-1)
        predicted = torch.argsort(output)[-num_candidates:]
        nxt = random.randint(1,scope)
        start = torch.cat([start,predicted[-nxt].reshape(1,-1).float()],1)
    return start,predicted,output

In [8]:
softmax = nn.Softmax(dim=0)

In [9]:
def showNext(t,num_candidates=10,ts=0.005):
    _,predicted,output = generate_seq(t,1,num_candidates)
    prob = softmax(output)
    scope = 0
    for i in range(num_candidates):
        if prob[predicted[num_candidates-i-1]]<ts:
            scope=num_candidates-i-1
#             print(scope)
            break
            
    #     print(t.int().cpu().numpy()[0])
    predicted = predicted[scope+1:].cpu().numpy().tolist()
    prob = prob[predicted].detach().cpu().numpy().tolist()
#     print("预测的序号排序:",end=' ')
#     print(predicted)
#     print("对应的可能性:",end=' ')
#     print(prob)
    return predicted,prob

In [332]:
seq = [21]
showNext(seq,ts=0.001)

[21, 11, 5, 30]

In [ ]:
t = torch.FloatTensor([0]).reshape(1,-1)
max_len = 60
pattern = set()
while t.size(1)<max_len:
    t,predicted,output = generate_seq(t,1,3,1)
    prob = softmax(output)
    print(t.int().cpu().numpy()[0])
    print("预测的序号排序:",end=' ')
    print(predicted)
    print("对应的可能性:",end=' ')
    print(prob[predicted])
    print()
    if 30 in t[0]:
        break
print(t.int().cpu().numpy()[0])
pattern.add(tuple(t.int().cpu().numpy()[0]))

In [166]:
concurrent_path = [i for i in seq]
concurrent_path_list = []
concorrent_tmp = list(concorrent_set)
for i,event in enumerate(concorrent_tmp):
    con_path = seq+[event]
    print(con_path)
    pre,_ = showNext(con_path)
    pre = set(pre)
    print(pre)
    concorrent_set.remove(event)
    while pre!=concorrent_set:
        for j in concorrent_set:
            pre.remove(j)
        con_path = con_path+[pre.pop()]
        pre,_ = showNext(con_path)
        pre = set(pre)
    concorrent_set.add(event)
    concurrent_path.extend(con_path[len(seq):])

[0, 5]
预测的序号排序: [22, 5]
对应的可能性: [0.24863998591899872, 0.7509623765945435]
{5, 22}
预测的序号排序: [22, 5]
对应的可能性: [0.34808650612831116, 0.6518101096153259]
预测的序号排序: [22]
对应的可能性: [0.9969695210456848]
[0, 22]
预测的序号排序: [5]
对应的可能性: [0.9984245300292969]
{5}


In [34]:
count_event = dict()
for session in sessions:
    event_set = list(set(session))
    for event in event_set:
        if event not in count_event:
            count_event[event]=0
        count_event[event]=count_event[event]+1
print(count_event)
for event in count_event:
    count_event[event] = count_event[event]/len(sessions)
print(count_event)

{0: 4855, 5: 4855, 9: 4855, 11: 4855, 21: 3976, 22: 4855, 23: 3960, 26: 4855, 30: 4855, 2: 721, 3: 1227, 4: 1153, 6: 17, 16: 17, 18: 17, 25: 17}
{0: 1.0, 5: 1.0, 9: 1.0, 11: 1.0, 21: 0.8189495365602472, 22: 1.0, 23: 0.815653964984552, 26: 1.0, 30: 1.0, 2: 0.14850669412976314, 3: 0.25272914521112255, 4: 0.23748712667353244, 6: 0.003501544799176107, 16: 0.003501544799176107, 18: 0.003501544799176107, 25: 0.003501544799176107}


In [64]:
def find_concurrent(seq):
    concorrent_set = set()
    concurrent_path_list= []
    pre_predict,prob = showNext(seq,ts=0.01)
    if seq[-1] in pre_predict:
        pre_predict.remove(seq[-1])
    if len(pre_predict)<2:
        return concorrent_set
    for i,event in enumerate(pre_predict):
        cur_seq = seq+[event]
        cur_predicted,prob = showNext(cur_seq,ts=0.01)
        for j in pre_predict[i+1:] :
            pre1,_ = showNext(seq+[j])
            if event in pre1 and j in cur_predicted:
                concorrent_set.add(event)  
                concorrent_set.add(j) 
    concurrent_path = [i for i in seq]
    concorrent_tmp = list(concorrent_set)
    for i,event in enumerate(concorrent_tmp):
        con_path = seq+[event]
        print(con_path)
        pre,_ = showNext(con_path)
        pre = set(pre)
        print(pre)
        concorrent_set.remove(event)
        while pre!=concorrent_set and not pre.issubset(concorrent_set):
            for j in concorrent_set:
                if j in pre:
                    pre.remove(j)
            con_path = con_path+[pre.pop()]
            pre,_ = showNext(con_path)
            pre = set(pre)
        concorrent_set.add(event)
        concurrent_path.extend(con_path[len(seq):])
        concurrent_path_list.append(con_path)
    merge,_ = showNext(concurrent_path)
    return concorrent_set,merge,concurrent_path_list

In [62]:
showNext([0,5,22,5,5])

预测的序号排序: [9, 26, 11]
对应的可能性: [0.01762796938419342, 0.036271460354328156, 0.9450459480285645]


([9, 26, 11], [0.01762796938419342, 0.036271460354328156, 0.9450459480285645])

In [65]:
find_concurrent([0,5,22,5,5])

预测的序号排序: [9, 26, 11]
对应的可能性: [0.01762796938419342, 0.036271460354328156, 0.9450459480285645]
预测的序号排序: [9, 26, 11]
对应的可能性: [0.05149395763874054, 0.3727625012397766, 0.575558602809906]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.006680076010525227, 0.4850876033306122, 0.5082016587257385]
预测的序号排序: [11, 9]
对应的可能性: [0.010778561234474182, 0.988262951374054]
预测的序号排序: [11, 26]
对应的可能性: [0.4850876033306122, 0.5082016587257385]
预测的序号排序: [11, 9]
对应的可能性: [0.010778561234474182, 0.988262951374054]
预测的序号排序: [11, 9]
对应的可能性: [0.010778561234474182, 0.988262951374054]
[0, 5, 22, 5, 5, 9]
预测的序号排序: [9, 26, 11]
对应的可能性: [0.05149395763874054, 0.3727625012397766, 0.575558602809906]
{9, 26, 11}
预测的序号排序: [9, 26, 11]
对应的可能性: [0.011504905298352242, 0.2595791518688202, 0.7285224199295044]
预测的序号排序: [26, 11]
对应的可能性: [0.09336795657873154, 0.9018100500106812]
[0, 5, 22, 5, 5, 26]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.006680076010525227, 0.4850876033306122, 0.5082016587257385]
{9, 26, 11}
预测的序号排序: [9, 11, 26]
对应的可能性: [0.01052768062800169,

({9, 11, 26},
 [11, 9],
 [[0, 5, 22, 5, 5, 9, 9, 9],
  [0, 5, 22, 5, 5, 26, 26, 26],
  [0, 5, 22, 5, 5, 11, 11, 11]])

In [177]:
def find_loop(seq):
    pass

In [178]:
def find_branch(seq):
    pass

In [68]:
seq = [11, 9, 11, 9, 11, 9, 26, 26, 26]
concorrent_set = set()
concurrent_path_list= []
pre_predict,prob = showNext(seq,ts=0.01)
if seq[-1] in pre_predict:
    pre_predict.remove(seq[-1])
for i,event in enumerate(pre_predict):
    cur_seq = seq+[event]
    cur_predicted,prob = showNext(cur_seq,ts=0.01)
    for j in pre_predict[i+1:]:
        pre1,_ = showNext(seq+[j])
        if event in pre1 and j in cur_predicted:
            concorrent_set.add(event)  
            concorrent_set.add(j) 
concurrent_path = [i for i in seq]


预测的序号排序: [2, 3, 4, 30, 23]
对应的可能性: [0.05369764193892479, 0.05526859685778618, 0.06050111725926399, 0.2377098798751831, 0.5877823829650879]
预测的序号排序: [2, 4, 3, 30, 23]
对应的可能性: [0.08384238183498383, 0.11713362485170364, 0.14755822718143463, 0.20119619369506836, 0.4497317969799042]
预测的序号排序: [2, 23, 4, 3]
对应的可能性: [0.028665073215961456, 0.033793333917856216, 0.3921320140361786, 0.5424069762229919]
预测的序号排序: [4, 3]
对应的可能性: [0.3911248743534088, 0.603672981262207]
预测的序号排序: [25, 11, 4, 2, 3, 30, 23]
对应的可能性: [0.005198594182729721, 0.020473534241318703, 0.05300910398364067, 0.08968948572874069, 0.1056489497423172, 0.34301844239234924, 0.3784100115299225]
预测的序号排序: [23]
对应的可能性: [0.9999033212661743]
预测的序号排序: [2, 23, 4, 3]
对应的可能性: [0.028665073215961456, 0.033793333917856216, 0.3921320140361786, 0.5424069762229919]
预测的序号排序: [4, 3]
对应的可能性: [0.3911248743534088, 0.603672981262207]
预测的序号排序: [25, 11, 4, 2, 3, 30, 23]
对应的可能性: [0.005198594182729721, 0.020473534241318703, 0.05300910398364067, 0.0896894857287406

In [70]:
concorrent_set.remove(30)

In [71]:
concorrent_tmp = list(concorrent_set)
for i,event in enumerate(concorrent_tmp):
    con_path = seq+[event]
    print("cur_path:",con_path)
    pre,_ = showNext(con_path)
    pre = set(pre)
    print(pre)
    concorrent_set.remove(event)
    while pre!=concorrent_set and not pre.issubset(concorrent_set):
        for j in concorrent_set:
            if j in pre:
                pre.remove(j)
        con_path = con_path+[pre.pop()]
        pre,_ = showNext(con_path)
        pre = set(pre)
    concorrent_set.add(event)
    concurrent_path.extend(con_path[len(seq):])
    concurrent_path_list.append(con_path)
merge,_ = showNext(concurrent_path)

cur_path: [11, 9, 11, 9, 11, 9, 26, 26, 26, 2]
预测的序号排序: [2, 4, 3, 30, 23]
对应的可能性: [0.08384238183498383, 0.11713362485170364, 0.14755822718143463, 0.20119619369506836, 0.4497317969799042]
{2, 3, 4, 23, 30}
预测的序号排序: [3, 23, 2, 4]
对应的可能性: [0.02893654815852642, 0.045643649995326996, 0.38048607110977173, 0.5404440760612488]
预测的序号排序: [18, 2, 3, 4]
对应的可能性: [0.007617548108100891, 0.012384964153170586, 0.021838082000613213, 0.9568265080451965]
预测的序号排序: [3, 2, 4, 23]
对应的可能性: [0.005741815082728863, 0.0421714261174202, 0.08599156141281128, 0.8649125099182129]
预测的序号排序: [3, 2, 23, 30, 4]
对应的可能性: [0.0055436501279473305, 0.00710405083373189, 0.025522660464048386, 0.02755546011030674, 0.9339141249656677]
预测的序号排序: [3, 30, 18, 23, 2, 4]
对应的可能性: [0.00868134293705225, 0.008983652107417583, 0.011883899569511414, 0.03815518319606781, 0.3577229678630829, 0.5738712549209595]
预测的序号排序: [18, 3, 23, 2, 4]
对应的可能性: [0.005704065319150686, 0.009191661141812801, 0.015055007301270962, 0.01958266645669937, 0.949583470821

预测的序号排序: [6, 23, 30, 5, 4, 3]
对应的可能性: [0.008524603210389614, 0.011513873934745789, 0.013502707704901695, 0.013898352161049843, 0.022668810561299324, 0.9280903935432434]
预测的序号排序: [4, 3, 23, 5]
对应的可能性: [0.011033434420824051, 0.12491286545991898, 0.35380470752716064, 0.5010837316513062]
预测的序号排序: [3, 23, 5]
对应的可能性: [0.024538634344935417, 0.10347607731819153, 0.8638625144958496]
预测的序号排序: [2, 23, 4, 3, 5]
对应的可能性: [0.01334332674741745, 0.014628302305936813, 0.05419773980975151, 0.1446402221918106, 0.7704848051071167]
预测的序号排序: [30, 2, 4, 3]
对应的可能性: [0.005858671385794878, 0.008532148785889149, 0.3840721547603607, 0.5989634990692139]
预测的序号排序: [30, 4, 3]
对应的可能性: [0.005984411109238863, 0.47227945923805237, 0.5205335021018982]
预测的序号排序: [30, 3, 4]
对应的可能性: [0.008617321960628033, 0.2426387071609497, 0.7486110329627991]
预测的序号排序: [30, 4, 3]
对应的可能性: [0.13855521380901337, 0.2444543093442917, 0.6167486906051636]
预测的序号排序: [4, 30, 3]
对应的可能性: [0.06648334860801697, 0.09203603118658066, 0.8411422967910767]
预测的序

预测的序号排序: [3, 4]
对应的可能性: [0.033352818340063095, 0.9665488600730896]
预测的序号排序: [3, 4]
对应的可能性: [0.033189352601766586, 0.9667131304740906]
预测的序号排序: [3, 4]
对应的可能性: [0.03303997591137886, 0.9668632745742798]
预测的序号排序: [3, 4]
对应的可能性: [0.032902851700782776, 0.9670009016990662]
预测的序号排序: [3, 4]
对应的可能性: [0.03277641162276268, 0.9671278595924377]
预测的序号排序: [3, 4]
对应的可能性: [0.032659418880939484, 0.9672455191612244]
预测的序号排序: [3, 4]
对应的可能性: [0.0325508750975132, 0.9673545956611633]
预测的序号排序: [3, 4]
对应的可能性: [0.03244968503713608, 0.9674562215805054]
预测的序号排序: [3, 4]
对应的可能性: [0.03235539793968201, 0.9675511121749878]
预测的序号排序: [3, 4]
对应的可能性: [0.032267119735479355, 0.9676398038864136]
预测的序号排序: [3, 4]
对应的可能性: [0.032184239476919174, 0.96772301197052]
预测的序号排序: [3, 4]
对应的可能性: [0.032106515020132065, 0.9678011536598206]
预测的序号排序: [3, 4]
对应的可能性: [0.032033346593379974, 0.9678747057914734]
预测的序号排序: [3, 4]
对应的可能性: [0.031964465975761414, 0.9679438471794128]
预测的序号排序: [3, 4]
对应的可能性: [0.031899593770504, 0.9680091142654419]
预测的序号排

KeyboardInterrupt: 

In [74]:
print(concurrent_path_list)

[[11, 9, 11, 9, 11, 9, 26, 26, 26, 2, 2, 2, 18, 2, 2, 2, 2, 2, 23, 23, 23, 21, 21, 21, 30, 18, 2, 30, 2, 21, 2, 21, 21, 2, 2, 2, 2, 2], [11, 9, 11, 9, 11, 9, 26, 26, 26, 3, 3, 3, 3, 3, 3, 3, 3, 3, 23, 23, 23, 21, 21, 21, 30, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]


In [66]:
showNext([26,26,26,23,23,23,21,21,21])

预测的序号排序: [30]
对应的可能性: [0.9997605681419373]


([30], [0.9997605681419373])

In [77]:
find_concurrent([22, 5, 5, 5])

预测的序号排序: [9, 11, 26]
对应的可能性: [0.01060451753437519, 0.3045552968978882, 0.6845642328262329]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.10653172433376312, 0.18710242211818695, 0.7063077688217163]
预测的序号排序: [9]
对应的可能性: [0.9990488886833191]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.008123203180730343, 0.06586132943630219, 0.92600017786026]
预测的序号排序: [9]
对应的可能性: [0.9990488886833191]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.008123203180730343, 0.06586132943630219, 0.92600017786026]
预测的序号排序: [11, 26]
对应的可能性: [0.06586132943630219, 0.92600017786026]
[22, 5, 5, 5, 9]
预测的序号排序: [9, 11, 26]
对应的可能性: [0.10653172433376312, 0.18710242211818695, 0.7063077688217163]
{9, 26, 11}
预测的序号排序: [9, 26, 11]
对应的可能性: [0.02290080487728119, 0.4846383333206177, 0.4920712411403656]
预测的序号排序: [9, 26, 11]
对应的可能性: [0.007098402362316847, 0.10752785205841064, 0.8851501941680908]
预测的序号排序: [9, 26, 11]
对应的可能性: [0.007510153576731682, 0.2285967767238617, 0.7633930444717407]
预测的序号排序: [11, 26]
对应的可能性: [0.04344955459237099, 0.9561227560043335]
[22, 5, 5, 5, 26]
预测

({9, 11, 26},
 [3, 30, 4, 26, 2, 23, 11],
 [[22, 5, 5, 5, 9, 9, 9, 9, 9], [22, 5, 5, 5, 26, 26, 26], [22, 5, 5, 5, 11]])

In [49]:
concorrent_set

{9, 11}

In [50]:
concorrent_set.issubset({1,9,11})

True

In [201]:
import itertools
 
def permutation(li):
  return [list(ele) for ele in (itertools.permutations(li))]
permutation([9,11,26])

[[9, 11, 26], [9, 26, 11], [11, 9, 26], [11, 26, 9], [26, 9, 11], [26, 11, 9]]

In [223]:
def generate_seq(start,window_size=10,num_candidates=5,scope=None):
    if isinstance(start,list):
        start = torch.FloatTensor(start).reshape(1,-1)
    bg = start.size(1) 
    if scope==None:
        scope=num_candidates
    for i in range(bg,bg+window_size):
#         start = torch.FloatTensor(start)
        seq = start.clone().detach().view(-1, i, input_size).to(device)
        output = model(seq).cpu()[:,-1,:]
        output = output.reshape(-1)
        predicted = torch.argsort(output)[-num_candidates:]
        nxt = random.randint(1,scope)
        start = torch.cat([start,predicted[-nxt].reshape(1,-1).float()],1)
    return start,predicted,output

In [377]:
def predict_next_event(seq,num_candidates,ts):
    _,predicted,output = generate_seq(seq,1,num_candidates)
    prob = softmax(output)
    scope = 0
    for i in range(num_candidates):
        if prob[predicted[num_candidates-i-1]]<ts:
            scope=num_candidates-i-1
            break
    return predicted[scope+1:].cpu().numpy().tolist()

def showNext(seq,num_candidates=10,ts=0.005,out_of_order = False,bg=0,ed=-1):
    res = []
    if not out_of_order:
        res.extend(predict_next_event(seq,num_candidates,ts))
    else:
        seq_list = permutation(seq[bg:ed])
        for s in seq_list:
            res.extend(predict_next_event(s+seq[ed:],num_candidates,ts))
        res = list(set(res))
    return res

In [345]:
showNext([9])

[9, 2, 4, 3, 23, 26, 11]

In [354]:
def isconnected(e1,e2,eventmap):
    eventList = list(eventmap.keys())
    visited = {i:False for i in eventList}
    que = []
    que.append(e1)
    while len(que)!=0:
        cur = que.pop(0)
        visited[cur] = True
        if e2 in eventmap[cur]:
            return True
        for e in eventmap[cur]:
            if e in eventList and not visited[e]:
                que.append(e)
    return False
        

In [355]:
next_event = [9,11,26]
concurrent_group=[]
if 30 in next_event:
    next_event.remove(30)
event_to_group = [-1 for _ in next_event]
event_to_next = {event:showNext([event]) for event in next_event}
for i,event in enumerate(next_event):
    if event_to_group[i]==-1:
        cur_group = [event]
        concurrent_group.append(cur_group)
        event_to_group[i] = len(concurrent_group)-1
    for j,event2 in enumerate(next_event[i+1:]):
        if isconnected(event,event2,event_to_next) and isconnected(event2,event,event_to_next):
            if event_to_group[j+i+1]==-1:
                event_to_group[j+i+1]=event_to_group[i]
                concurrent_group[event_to_group[i]].append(event2)
#                 print(concurrent_group,i,j+i+1)

In [358]:
def recognize_branch(seq,next_event,concurrent=False):
    concurrent_group=[]
    if 30 in next_event:
        next_event.remove(30)
    event_to_group = [-1 for _ in next_event]
    event_to_next = {event:showNext([event]) for event in next_event}
    for i,event in enumerate(next_event):
        if event_to_group[i]==-1:
            cur_group = [event]
            concurrent_group.append(cur_group)
            event_to_group[i] = len(concurrent_group)-1
        for j,event2 in enumerate(next_event[i+1:]):
            if isconnected(event,event2,event_to_next) and isconnected(event2,event,event_to_next):
                if event_to_group[j+i+1]==-1:
                    event_to_group[j+i+1]=event_to_group[i]
                    concurrent_group[event_to_group[i]].append(event2)
    #                 print(concurrent_group,i,j+i+1)
    return concurrent_group

In [359]:
concurrent_group

[[9, 11, 26]]

In [319]:
def recognize_branch(seq=[],next_event,concurrent=False):
    concurrent_group=[]
    event_to_group = [-1 for _ in next_event]
#     event_to_next = {event:showNext(cur_seq)[0] for event in next_event}
    for i,event in enumerate(next_event):
        if event==30:
            continue
        cur_seq = seq+[event]
        cur_group = [event]
        if event_to_group[i]==-1:
            concurrent_group.append(cur_group)
            event_to_group[i] = len(concurrent_group)-1
        cur_predicted = showNext(cur_seq,out_of_order=concurrent)
        for j,event2 in enumerate(next_event[i+1:]):
            if event2==30:
                continue
            pre1= showNext(seq+[event2],out_of_order=concurrent)
            if event in pre1 and event2 in cur_predicted:
                if event_to_group[j+i+1]==-1:
                    event_to_group[j+i+1]=event_to_group[i]
                    concurrent_group[event_to_group[i]].append(event2)
                    print(concurrent_group,i,j+i+1)
    return concurrent_group

In [ ]:
def recognize_branch(seq,next_event,concurrent=False):
    concurrent_group=[]
    if 30 in next_event:
        next_event.remove(30)
    event_to_group = [-1 for _ in next_event]
    event_to_next = {event:showNext([event]) for event in next_event}
    for i,event in enumerate(next_event):
        if event_to_group[i]==-1:
            cur_group = [event]
            concurrent_group.append(cur_group)
            event_to_group[i] = len(concurrent_group)-1
        for j,event2 in enumerate(next_event[i+1:]):
            if isconnected(event,event2,event_to_next) and isconnected(event2,event,event_to_next):
                if event_to_group[j+i+1]==-1:
                    event_to_group[j+i+1]=event_to_group[i]
                    concurrent_group[event_to_group[i]].append(event2)
    #                 print(concurrent_group,i,j+i+1)
    return concurrent_group

In [336]:
showNext([23])

[26, 5, 11, 21, 23]

In [361]:
recognize_branch([], [11,9,26])

[[11, 9, 26]]

In [269]:
showNext([5,22],out_of_order=True,ed=2)

[9, 26, 11, 5]

In [379]:
def workflow_construction(seq=[0],end={30},concurrent=False):
    
    next_event= showNext(seq,out_of_order=concurrent,ed=len(seq))
    for event in seq:
        if event in next_event:
            next_event.remove(event)
    print(seq,"下一个event",next_event)
#     while seq[-1] in next_event:
#         seq =seq+[seq[-1]]
#         next_event,_ = showNext(seq)
    if set(next_event)==end or set(next_event).issubset(end):
        return seq
    if len(next_event)>=2:
        print("begin find concurrent",seq,next_event)
        concorrent_group = recognize_branch([],next_event,concurrent)
        print("find concurrent",seq,concorrent_group)
    else:
        concorrent_group = [[next_event[0]]]
    for group in concorrent_group:
        if len(group)>=2:
            print("合并",seq,group)
            concurrent_path = []
            for event in group:
                event_set = set(group)
                event_set.remove(event)
                new_seq = workflow_construction(seq+[event],event_set)
                event_set.add(event)
                concurrent_path.extend(new_seq[len(seq):])
            print("完成合并",concurrent_path,end)
            workflow_construction(concurrent_path,end,True)
        else:
            workflow_construction(seq+[group[0]],{30})
    return seq

In [372]:
def workflow_construction(seq=[0],end={30},concurrent=False):
    
    next_event= showNext(seq,out_of_order=concurrent,ed=len(seq))
    for event in seq:
        if event in next_event:
            next_event.remove(event)
    print(seq,"下一个event",next_event)
#     while seq[-1] in next_event:
#         seq =seq+[seq[-1]]
#         next_event,_ = showNext(seq)
    if set(next_event)==end or set(next_event).issubset(end):
        return seq
    if len(next_event)>=2:
        print("begin find concurrent",seq,next_event)
        concorrent_group = recognize_branch([],next_event,concurrent)
        print("find concurrent",seq,concorrent_group)
    else:
        concorrent_group = [[next_event[0]]]
    for group in concorrent_group:
        if len(group)>=2:
            print("合并",seq,group)
#             concurrent_path = []
#             for event in group:
#                 event_set = set(group)
#                 event_set.remove(event)
#                 new_seq = workflow_construction(seq+[event],event_set)
#                 event_set.add(event)
#                 concurrent_path.extend(new_seq[len(seq):])
#             print("完成合并",concurrent_path,end)
            workflow_construction(group,end,True)
        else:
            workflow_construction(group,{30})
    return seq

In [387]:
showNext([22, 9, 9,11,11,  11, 26, 26, 26] )

[26, 4, 3, 30, 2, 23, 11]

In [383]:
def find_merge_point(seq,concorrent_group):
    concorrent_set = set(concorrent_group)
    concurrent_path = [i for i in seq]
    for i,event in enumerate(concorrent_group):
        con_path = seq+[event]
        # print("cur_path:",con_path)
        pre = showNext(con_path)
        pre = set(pre)
        # print(pre)
        concorrent_set.remove(event)
        while pre!=concorrent_set and not pre.issubset(concorrent_set):
            for j in concorrent_set:
                if j in pre:
                    pre.remove(j)
            con_path = con_path+[pre.pop()]
            pre = showNext(con_path)
            pre = set(pre)
        concorrent_set.add(event)
        concurrent_path.extend(con_path[len(seq):])
    merge = showNext(concurrent_path)
    return concurrent_path,merge

In [384]:
find_merge_point([5,22],set([9,26,11]))

([5, 22, 9, 9, 9, 9, 26, 26, 26, 11], [30, 9, 11])

In [380]:
print(workflow_construction([0],{30}))

[0] 下一个event [22, 5]
begin find concurrent [0] [22, 5]
find concurrent [0] [[22, 5]]
合并 [0] [22, 5]
[0, 22] 下一个event [5]
[0, 5] 下一个event [22]
完成合并 [22, 5] {30}
[22, 5] 下一个event [9, 26, 11]
begin find concurrent [22, 5] [9, 26, 11]
find concurrent [22, 5] [[9, 26, 11]]
合并 [22, 5] [9, 26, 11]
[22, 5, 9] 下一个event [26, 11]
[22, 5, 26] 下一个event [21, 9, 11]
begin find concurrent [22, 5, 26] [21, 9, 11]
find concurrent [22, 5, 26] [[21], [9, 11]]
[22, 5, 26, 21] 下一个event [9, 11]
begin find concurrent [22, 5, 26, 21] [9, 11]
find concurrent [22, 5, 26, 21] [[9, 11]]
合并 [22, 5, 26, 21] [9, 11]
[22, 5, 26, 21, 9] 下一个event [11]
[22, 5, 26, 21, 11] 下一个event [9]
完成合并 [9, 11] {30}
[9, 11] 下一个event [2, 3, 4, 23, 26]
begin find concurrent [9, 11] [2, 3, 4, 23, 26]
find concurrent [9, 11] [[2, 3, 4], [23], [26]]
合并 [9, 11] [2, 3, 4]
[9, 11, 2] 下一个event [30, 23, 3, 4]
begin find concurrent [9, 11, 2] [30, 23, 3, 4]
find concurrent [9, 11, 2] [[23], [3, 4]]
[9, 11, 2, 23] 下一个event []
合并 [9, 11, 2] [3, 4]

find concurrent [6, 16, 5, 22, 23, 21, 9, 2, 4] [[3]]
[6, 16, 5, 22, 23, 21, 9, 2, 4, 3] 下一个event [30]
完成合并 [3, 4] {3, 4}
[3, 4] 下一个event [2, 23]
begin find concurrent [3, 4] [2, 23]
find concurrent [3, 4] [[2], [23]]
[3, 4, 2] 下一个event [23]
[3, 4, 2, 23] 下一个event []
[3, 4, 23] 下一个event []
[6, 16, 5, 22, 23, 21, 9, 4] 下一个event [2, 26, 11, 3]
begin find concurrent [6, 16, 5, 22, 23, 21, 9, 4] [2, 26, 11, 3]
find concurrent [6, 16, 5, 22, 23, 21, 9, 4] [[2, 3], [26], [11]]
合并 [6, 16, 5, 22, 23, 21, 9, 4] [2, 3]
[6, 16, 5, 22, 23, 21, 9, 4, 2] 下一个event []
[6, 16, 5, 22, 23, 21, 9, 4, 3] 下一个event []
完成合并 [2, 3] {2, 3}
[2, 3] 下一个event [4, 23, 30]
begin find concurrent [2, 3] [4, 23, 30]
find concurrent [2, 3] [[4], [23]]
[2, 3, 4] 下一个event [23]
[2, 3, 4, 23] 下一个event []
[2, 3, 23] 下一个event []
[6, 16, 5, 22, 23, 21, 9, 4, 26] 下一个event [3, 30]
begin find concurrent [6, 16, 5, 22, 23, 21, 9, 4, 26] [3, 30]
find concurrent [6, 16, 5, 22, 23, 21, 9, 4, 26] [[3]]


KeyboardInterrupt: 

In [132]:
find_merge_point([0],{22,5})

预测的序号排序: [22, 5]
对应的可能性: [0.2932291626930237, 0.705966591835022]
预测的序号排序: [22, 5]
对应的可能性: [0.42156782746315, 0.5777947306632996]
预测的序号排序: [22]
对应的可能性: [0.9989638328552246]
预测的序号排序: [5]
对应的可能性: [0.9994300007820129]
预测的序号排序: [26, 9, 11]
对应的可能性: [0.015241644345223904, 0.021479245275259018, 0.9622746109962463]


([0, 5, 5, 5, 22], [26, 9, 11])